# Урок 3. Системы управления базами данных MongoDB и SQLite в Python
### 1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию,  записывающую собранные вакансии в созданную БД.

pip install pymongo

In [1]:
import pandas as pd
from pymongo import MongoClient

In [2]:
DATASET_PATH_HH = "vacancy_hh_old.csv"
DATASET_PATH_SJ = "vacancy_sj_old.csv"

In [3]:
df_hh = pd.read_csv(DATASET_PATH_HH)
df_sj = pd.read_csv(DATASET_PATH_SJ)
df_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     40 non-null     object
 1   link     40 non-null     object
 2   sum_min  40 non-null     int64 
 3   sum_max  40 non-null     int64 
 4   firm     40 non-null     object
 5   adress   40 non-null     object
 6   site     40 non-null     object
dtypes: int64(2), object(5)
memory usage: 2.3+ KB


In [4]:
# инициализация базы и коллекции
client = MongoClient('localhost', 27017)
db = client['vacancy_db']
mycollection = db.vacancy_collection

In [5]:
#загрузить данные с hh.ru в базу
df_hh.reset_index(inplace=True)
data_dict = df_hh.to_dict("records")
mycollection.insert_many(data_dict)

In [6]:
#загрузить данные с sj.ru в базу
df_sj.reset_index(inplace=True)
data_dict = df_sj.to_dict("records")
mycollection.insert_many(data_dict)

### 2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы.

In [7]:
#найти записи больше равно введенной суммы
input_sum = int(input("Введите минимальный размер заработной платы:"))
cursor = db['vacancy_collection'].find({'sum_min': { '$gt': input_sum}})
df =  pd.DataFrame(list(cursor))
df

Введите минимальный размер заработной платы:100000


,_id,index,name,link,sum_min,sum_max,firm,adress,site
0,613e5283ca8ce46c01b9021a,9,Junior Data Engineer (Python),https://hh.ru/vacancy/47018981?from=vacancy_se...,120000.0,120000.0,DataFork,"Москва, Выставочная и еще 2",hh.ru
1,613e5283ca8ce46c01b9021d,12,Data scientist,https://hh.ru/vacancy/44989370?from=vacancy_se...,200000.0,0.0,Bip.ru,"Москва, Бауманская",hh.ru
2,613e5283ca8ce46c01b90220,15,Middle Data Scientist / Computer Vision Engineer,https://hh.ru/vacancy/47622728?from=vacancy_se...,130000.0,200000.0,ZennoLab,Москва,hh.ru
3,613e5283ca8ce46c01b90222,17,Data Analyst,https://hh.ru/vacancy/46956576?from=vacancy_se...,200000.0,0.0,Новакид РУС,Москва,hh.ru
4,613e5283ca8ce46c01b90223,18,Data Engineer,https://hh.ru/vacancy/46976609?from=vacancy_se...,300000.0,0.0,Храмцова Полина,Москва,hh.ru
5,613e5283ca8ce46c01b90224,19,Системный аналитик\ data scientist,https://hh.ru/vacancy/47688264?from=vacancy_se...,190000.0,250000.0,HFLabs,Москва,hh.ru
6,613e5283ca8ce46c01b90225,20,Специалист Computer Vision (Senior Data Scient...,https://hh.ru/vacancy/47488734?from=vacancy_se...,180000.0,300000.0,ООО Медицинские Скрининг Системы,Санкт-Петербург,hh.ru
7,613e5283ca8ce46c01b90227,22,Data Analyst,https://hh.ru/vacancy/46956577?from=vacancy_se...,200000.0,0.0,Новакид РУС,Санкт-Петербург,hh.ru
8,613e5283ca8ce46c01b90239,0,Backend Python Django Developer Data Science C...,https://www.superjob.ru/vakansii/backend-pytho...,150000.0,175000.0,Longevity InTime,Москва,superjob.ru
9,613e5283ca8ce46c01b90249,16,Системный аналитик,https://www.superjob.ru/vakansii/sistemnyj-ana...,120000.0,NaN,Эссистэнс Сервисес,Москва,superjob.ru


### 3. Написать функцию, которая будет добавлять в вашу базу данных только новые вакансии с сайта.

In [8]:
def load_set(dt, db_mongodb, db_col):
    # соединениес базой MongoDB
    client = MongoClient('localhost', 27017)
    db = client[db_mongodb] 
    
    # загрузить записи из базы в dataset Pandas
    cursor = db[db_col].find()
    dfmongo =  pd.DataFrame(list(cursor))            
    dfmongo.to_csv('dfmongo.csv', index=False, encoding="utf-8")
    
    # проверяем два датасета на совпадение данных с базой по link
    test_df = pd.merge(dt, dfmongo, on = 'link', how = 'left')
    test_df
    
    #выбираем только новые
    test_df = test_df[test_df["site_y"].isnull()]
   
    if test_df['link'].count() > 0:
        new_df = test_df.drop(columns = ['_id', 'index', 'name_y','sum_min_y','sum_max_y','firm_y',
                                         'adress_y','site_y'])
        new_df.columns=['name', 'link', 'sum_min', 'sum_max', 'firm', 'adress', 'site']
        #записываем новые данные в базу
        new_df.reset_index(inplace=True)
        data_dict = new_df.to_dict("records")
        mycollection.insert_many(data_dict)
        print(f"Записаны новые данные в файл: {test_df['link'].count()} шт")
    else:
        print(f"Новых данных нет")    
    

In [9]:
#подгружаем из файла новые данные по вакансии
DATASET_PATH_HH = "vacancy_hh.csv"
DATASET_PATH_SJ = "vacancy_sj.csv"
df_hh = pd.read_csv(DATASET_PATH_HH)
df_sj = pd.read_csv(DATASET_PATH_SJ)

In [10]:
df_hh.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   name     60 non-null     object
 1   link     60 non-null     object
 2   sum_min  60 non-null     int64 
 3   sum_max  60 non-null     int64 
 4   firm     60 non-null     object
 5   adress   60 non-null     object
 6   site     60 non-null     object
dtypes: int64(2), object(5)
memory usage: 3.4+ KB


In [11]:
df_sj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 7 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   name     38 non-null     object 
 1   link     38 non-null     object 
 2   sum_min  6 non-null      float64
 3   sum_max  5 non-null      float64
 4   firm     36 non-null     object 
 5   adress   38 non-null     object 
 6   site     38 non-null     object 
dtypes: float64(2), object(5)
memory usage: 2.2+ KB


In [12]:
load_set(df_hh,'vacancy_db','vacancy_collection')

Записаны новые данные в файл: 40 шт


In [13]:
load_set(df_sj,'vacancy_db','vacancy_collection')

Записаны новые данные в файл: 12 шт


#Удаление записей для теста
mycollection.drop()